In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')
s2p_write_folder = os.path.join('/home/mplitt','2P_scratch')

mouse = '4467975.3'

file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':2,'scan':7},
             {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
             {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':6},
             {'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':6},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             {'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':4},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':8},
             )

In [3]:
def check_and_make_h5s(file_list,runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus2','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [4]:
h5list_all, frs_all = check_and_make_h5s(file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': h5list_all,
                               'save_path0': os.path.join(write_folder,mouse,"YMazeAll"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':frs_all[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'saveNWB':True,
                                'threshold_scaling':.9,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4}
{'date': '29_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4}
{'date': '30_09_2020', 'scene': 'YMaze_LNovel', 'session': 2, 'scan': 7}
{'date': '01_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 7}
{'date': '02_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '03_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 6}
{'date': '04_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 6}
{'date': '05_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3}
{'date': '06_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4}
{'date': '06_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 8}
['/mnt/BigDisk/2P_scratch/4467975.3/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_004', '/mnt/BigDisk/2P_scratch/4467975.3/29_09_2020/YMaze_LNovel/YMaze_LNovel_001_004', '/mnt/Bi

In [4]:
# _file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
#              {'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
#              {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':2,'scan':7},
#              {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
#              {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
_file_list = ({'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':6},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             )

for f in _file_list:
    _h5list,_frs = check_and_make_h5s([f,])
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':1.,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 
    

{'date': '03_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 6}
['/mnt/BigDisk/2P_scratch/4467975.3/03_10_2020/YMaze_LNovel/YMaze_LNovel_001_006']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/mnt/BigDisk/2P_scratch/4467975.3/03_10_2020/YMaze_LNovel/YMaze_LNovel_001_006/YMaze_LNovel_001_006.h5']
time 1277.75 sec. Wrote 38928 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 38928 frames
NOTE: estimated bidiphase offset from data: 0 pixels
Reference frame, 22.59 sec.
Registered 2000/38928 in 93.56s
Registered 4000/38928 in 186.56s
Registered 6000/38928 in 280.32s
Registered 8000/38928 in 379.04s
Registered 10000/38928 in 478.14s
Registered 12000/38928 in 574.72s
Registered 14000/38928 in 664.06s
Registered 16000/38928 in 754.54s
Registered 18000/38928 in 855.

In [5]:
_file_list = ({'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':6},)

_h5list,_frs = check_and_make_h5s(_file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"04_10_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'threshold_scaling':1.,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)
for h5 in _h5list:
    !rm {h5}/*.h5 

{'date': '04_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 10532
{'date': '04_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 6}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 27315
['/mnt/BigDisk/2P_scratch/4467975.3/04_10_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2P_scratch/4467975.3/04_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_006']
{}
h5
** Found 2 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/mnt/BigDisk/2P_scratch/4467975.3/04_10_2020/YMaze_LNovel/YMaze_LNovel_001_003/YMaze_LNovel_001_003.h5', '/mnt/BigDisk/2P_scratch/4467975.3/04_10_2020/YMaz

In [6]:
_file_list = ({'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':4},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':8},)

_h5list,_frs = check_and_make_h5s(_file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"06_10_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'threshold_scaling':1.,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)
for h5 in _h5list:
    !rm {h5}/*.h5 

{'date': '06_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 10638
{'date': '06_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 8}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 28341
['/mnt/BigDisk/2P_scratch/4467975.3/06_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_004', '/mnt/BigDisk/2P_scratch/4467975.3/06_10_2020/YMaze_LNovel/YMaze_LNovel_001_008']
{}
h5
** Found 2 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/mnt/BigDisk/2P_scratch/4467975.3/06_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_004/YMaze_RewardReversal_001_004.h5', '/mnt/BigDisk/

In [5]:
for h5 in h5list_all:
    !rm {h5}/*.h5 